<a href="https://colab.research.google.com/github/dldbsrn0120/Sejong_AI_Final/blob/master/FinalProject_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AI Final Projcet 15010963 Lee Yoon Gu

# Implement Deep Learning Training Model using Keras, VGG16 CNN architecture

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# load libraries

import keras

from keras_applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint

from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers
from keras import optimizers, initializers, regularizers, metrics

import os
import random
import math


Using TensorFlow backend.


In [ ]:
# set VGG16 model
# do transter learning by using imagenet weights
pre_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), backend=keras.backend,
                  layers=keras.layers, models=keras.models, utils=keras.utils)
pre_model.trainable = True
pre_model.summary()

vgg_model = models.Sequential()
vgg_model.add(pre_model)
vgg_model.add(layers.Flatten())

# modify FC layers for classify 1000 -> 1024 as relu, and add 4 softmax
vgg_model.add(layers.Dense(4096, activation='relu'))
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dense(4, activation='softmax'))
# set soft max = 4 because i will classify 4 car classes.

# 1 - small car
# 2 - sedan
# 3 - suv
# 4 - van

# print summary
vgg_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:

# Loss function = categorical_cross entropy
# optimization function use = RMSprop
# set learning rate = 2e - 5
vgg_model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

#load images

train_dir = "/content/drive/My Drive/AI_Final/car_train_set/"
val_dir = "/content/drive/My Drive/AI_Final/car_valid_set/"

# rescale values from 0 to 255 for 0 to 1 rescaled values

train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

# set batchsize = 32

# load images by ImageDataGenerator

train_generator = train_datagen.flow_from_directory(train_dir, batch_size=32, target_size=(224, 224), color_mode='rgb')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=32, target_size=(224, 224), color_mode='rgb')

Found 1708 images belonging to 4 classes.
Found 571 images belonging to 4 classes.


In [ ]:
checkpoint = ModelCheckpoint(filepath="/content/drive/My Drive/AI_Final/car_weight.hdf5",
                             monitor='loss',
                             mode='min',
                             save_best_only=True
                             )

#do transfer learning by using imagenet

history = vgg_model.fit_generator(train_generator,
                                  steps_per_epoch=math.ceil(train_generator.n / train_generator.batch_size),
                                  epochs = 5, # 5 epoch (train all data by 5 times)
                                  validation_data=val_generator,
                                  validation_steps=math.ceil(val_generator.n / val_generator.batch_size),
                                  callbacks=[checkpoint],
                                  verbose=1)
# save result to my drive
vgg_model.save_weights("/content/drive/My Drive/AI_Final/car_weight_result.h5")
print("Done")

Epoch 1/5
53/54 [============================>.] - ETA: 15s - loss: 0.7518 - acc: 0.7029

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 16 could not be retrieved. It could be because a worker has died.
  UserWarning)


54/54 [==============================] - 966s 18s/step - loss: 0.7444 - acc: 0.7055 - val_loss: 0.2584 - val_acc: 0.8984
Epoch 2/5
54/54 [==============================] - 140s 3s/step - loss: 0.0960 - acc: 0.9696 - val_loss: 0.0225 - val_acc: 0.9912
Epoch 3/5
54/54 [==============================] - 145s 3s/step - loss: 0.0834 - acc: 0.9795 - val_loss: 0.0050 - val_acc: 0.9947
Epoch 4/5
54/54 [==============================] - 143s 3s/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.2435 - val_acc: 0.9895
Epoch 5/5
54/54 [==============================] - 142s 3s/step - loss: 0.0484 - acc: 0.9865 - val_loss: 0.0256 - val_acc: 0.9737
Done
